In [2]:
import Pkg
# Pkg.add("BSplineKit")
Pkg.add("Dierckx")
Pkg.add("Convex")
Pkg.add("FiniteDiff")
Pkg.add("ForwardDiff")
Pkg.add("Interpolations")
Pkg.add("StaticArrays")
Pkg.add("GeometryBasics")
Pkg.activate(@__DIR__)

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx
import ECOS
# using BSplineKit
using Dierckx
using Interpolations
using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using MeshCat
const mc = MeshCat
using StaticArrays
using Printf
using GeometryBasics

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
┌ Warning: The active manifest file at `/Users/jackhe/16745/final_proj/Manifest.toml` has an old format that is being maintained.
│ To update to the new format, which is supported by Julia versions ≥ 1.6.2, run `import Pkg; Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
│ To then record the julia version re-resolve with `Pkg.resolve()` and if there are resolve conflicts consider `Pkg.update()`.
└ @ Pkg.Types ~/.julia/juliaup/julia-1.10.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.10/Pkg/src/manifest.jl:318
  No Changes to `~/16745/final_proj/Project.toml`
  No Changes to `~/16745/final_proj/Manifest.toml`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ ~/16745/final_proj/Manifest.toml:0
   Resolving package versions...
  No Changes to `~/16745/final_proj/P

In [3]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [4]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 4 virtual cores)


In [11]:
function dynamics(model::NamedTuple, x::Vector, u::Vector)::Vector
    # kinematic bicycle model continuous time dynamics
    # lr = distance from rear axle to COM
    # L = wheelbase

    lr, L = model.lr, model.L

    # along track, cross track, heading w.r.t track, steering angle, speed
    s, r, e_psi, δ, v = x

    # rate of steer, acceleration
    δdot, a = u

    # side slip angle
    β = atan(lr/L * tan(δ))

    # curvature of reftraj
    # unit tangent
    T(_s) = reftraj_deriv(_s) / norm(reftraj_deriv(_s))
    Tdot = FD.derivative(T, s)
    # @show norm(reftraj_deriv(s))
    κ = norm(Tdot) / norm(reftraj_deriv(s))
    
    sin_e_psi, cos_e_psi = sincos(e_psi)

    sdot = v * cos_e_psi / (1 - r * κ)
    rdot = v * sin_e_psi

    β_dot = FD.derivative(_δ -> atan(lr/L * tan(_δ)), δ) * δdot
    psi_dot = v/lr * sin(β)

    # side slip angle rate + heading rate
    e_psi_dot = β_dot + psi_dot - sdot * κ
    # @show κ

    xdot = [
        sdot,
        rdot,
        e_psi_dot,
        δdot,
        a
    ]
    return xdot
end

function rk4(model::NamedTuple, ode::Function, x::Vector, u::Vector, dt::Real)::Vector
    k1 = dt * ode(model, x,        u)
    k2 = dt * ode(model, x + k1/2, u)
    k3 = dt * ode(model, x + k2/2, u)
    k4 = dt * ode(model, x + k3,   u)
    return x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

function hermite_simpson(model::NamedTuple, ode::Function, x1::Vector, x2::Vector, u::Vector, dt::Float64)::Vector
    ẋ1 = ode(model, x1, u)
    ẋ2 = ode(model, x2, u)
    x_half = 0.5 * (x1 + x2) + dt/8 * (ẋ1 - ẋ2)
    ẋ_half = ode(model, x_half, u)
    return x1 + dt/6 * (ẋ1 + 4 * ẋ_half + ẋ2) - x2
end

hermite_simpson (generic function with 1 method)

In [12]:
# X_mat = [0.0 0; 5 1; 10 1.5; 20 2; 30 1.5; 40 1; 50 0; 60 10; 70 20]

xs = 0.0:1:100
ys = 3 * cos.(0.1 * xs)
X_mat = hcat(xs, ys)

X = [X_mat[i,:] for i in 1:size(X_mat,1)]

itp = Interpolations.extrapolate(
    Interpolations.interpolate(
        X_mat,
        (
            BSpline(Cubic(Interpolations.Natural(OnGrid()))), 
            NoInterp()
        )
    ),
    Interpolations.Line()
)

function reftraj(s::Real)
    i = get_idx_from_dist(s)
    return [itp(i, 1); itp(i, 2)]
end
    
function reftraj_deriv(s::Real)
    return FD.derivative(reftraj, s)
end

function reftraj_2nd_deriv(s::Real)
    return FD.derivative(reftraj_deriv, s)
end

function reftraj_i(i::Real)
    return [itp(i, 1); itp(i, 2)]
end
    
function reftraj_deriv_i(i::Real)
    return FD.derivative(reftraj_i, i)
end

function reftraj_2nd_deriv_i(i::Real)
    return FD.derivative(reftraj_deriv_i, i)
end


"""
generate a vector of distances along the reftraj,
returns the vector of distances and the associated vector of indices
"""
function generate_distances()
    dt = 0.01
    ts = 0.0:dt:size(X, 1)
    drdts = copy(reftraj_deriv_i.(ts))
    print("drdts shape", size(drdts))
    drdts = [drdts[i, :] for i in 1:size(drdts, 1)]
    speeds = norm.(eachrow(drdts))
    distances = zeros(size(speeds, 1))
    distances[1] = speeds[1] * dt
    for i = 2:size(drdts, 1)
        distances[i] = speeds[i] * dt + distances[i - 1]
    end

    return distances, ts
end

distances, ts = generate_distances()
# @show distances, ts

# 1-indexed, so add 1 since ts are 0-indexed
function get_idx_from_dist(_d)
    return 1 + linear_interpolation(
        distances,
        ts,
        extrapolation_bc=Interpolations.Line())(_d)
end

function get_dist_from_idx(_i)
    return linear_interpolation(
        ts,
        distances,
        extrapolation_bc=Interpolations.Line())(_i - 1)
end

function get_pos_from_dist(_d)
    return reftraj(_d)
end

function get_dist_from_pos(x)
    # compute distance to each knot pt
    dists_to_path = norm.(X .- Ref(x))
    knot_idx = argmin(dists_to_path)
    # subsample the distance function in the interval [knot idx - 1, knot idx + 1]
    # for each subsample along the path, compute distance from x  to subsample
    path_subsample = reftraj_i.(knot_idx-1:0.1:knot_idx+1)
    dists_to_subsample = norm.(path_subsample .- Ref(x))
    knot_idx_subsample = argmin(dists_to_subsample) * 0.1 + (knot_idx - 1)
    # @show "subsample range", knot_idx-1:0.01:knot_idx+1
    # @show "spline position along subsample range", path_subsample
    # @show "subsampled dists" dists_to_subsample
    # @show "closest idx", knot_idx
    # @show "closest subsample idx", knot_idx_subsample
    return get_dist_from_idx(knot_idx_subsample)
end

function get_xy(s, r)
    closest_pt = reftraj(s)
    # take the unit normal vector, multiply by r 
    # and add to closest pt
    tangent = reftraj_deriv(s)
    unit_tangent = tangent / norm(tangent)
    unit_normal = [-unit_tangent[2]; unit_tangent[1]]
    return closest_pt + r * unit_normal
end

function get_sr(x, y)
    s = get_dist_from_pos([x, y])
    closest_pt = reftraj(s)

    path_to_xy = closest_pt - [x; y]
    r_mag = norm(path_to_xy)

    tangent = reftraj_deriv(s)
    cross_product = (tangent[1] * path_to_xy[2]) - (tangent[2] * path_to_xy[1])
    r = sign(cross_product) * r_mag
    return [s; r]
end


function matrix_to_vec(mat)
    vec = [mat[i,:] for i in 1:size(mat,1)]
    return vec
end


drdts shape(10101,)

matrix_to_vec (generic function with 1 method)

In [ ]:
# adapted from HW4
function create_idx(nx,nu,N)
    # create idx for indexing convenience
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    # and stacked dynamics constraints of size nx are
    # c[idx.c[i]] = <dynamics constraint at time step i>

    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]

    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx

    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function dynamics_constraints(params, Z)
    idx = params.idx
    N = params.N
    c = zeros(eltype(Z), idx.nc)
    model = params.model
    dt = params.dt

    for i = 1:N-1
        xi = Z[idx.x[i]]
        xi1 = Z[idx.x[i + 1]]
        ui = Z[idx.u[i]]
        c[idx.c[i]] = hermite_simpson(model, dynamics, xi, xi1, ui, dt) 
    end

    return c
end

function equality_constraints(params, Z)
    idx, xi = params.idx, params.xi
    init_constraint = xi - Z[idx.x[1]]
    return [init_constraint; dynamics_constraints(params, Z)]
end

function inequality_constraints(params, Z)
    return [0]
end

function cost(params::NamedTuple, Z::Vector)::Real
    # cost function
    idx, N, s_goal = params.idx, params.N, params.s_goal
    Q, Qf, R = params.Q, params.Qf, params.R

    J = 0
    for i = 1:N-1
        Xe = Z[idx.x[i]] - [s_goal / params.t_horizon * i, 0, 0, 0, 0]
        U = Z[idx.u[i]]
        # J += Xe' * Q * Xe + U' * R * U
        x = Z[idx.x[i]]
        J += U' * R * U + norm(get_xy(s_goal, 0) .- get_xy(x[1], x[2]))
    end

    # Xfe = Z[idx.x[N]] - [s_goal / params.t_horizon * N, 0, 0, 0, 0]
    # J += Xfe' * Qf * Xfe
    
    x = Z[idx.x[N]]
    J += norm(get_xy(s_goal, 0) .- get_xy(x[1], x[2]))

    return 0.5 * J
end

function optimize(Xpath, xi)
    dt = 0.5 #s
    t_horizon = 2.0 #s
    N = Int(t_horizon / dt)

    model = (
        L = 1.2,
        lr = 0.4,
    )

    track = (width = 3.0)
    idx = create_idx(5, 2, N)

    # mostly care about progress
    Q = diagm([1.0; 0; 0; 0; 0])
    Qf = diagm([10000.0; 0; 0; 0; 0])
    R = diagm([0.1; 0.1])
    
    v0 = 13.0
    pathlen = get_dist_from_idx(length(Xpath))
    s_goal = xi[1] + t_horizon * v0 * 1.1

    z0 = zeros(idx.nz)
    # TODO: if this isn't good enough, initialize steering angle using curvature of reference

    # initialize velocity by assuming constant velocity and that we reach the end of the track
    # at the end of the horizon

    # since we initialize the state with reftraj, e_psi is always 0. Since we assume const vel, 
    # v is always path len / time horizon.

    ts = 0.0:dt:t_horizon
    dists = xi[1] .+ (ts .* v0)

    
    for i = 1:N
        z0[idx.x[i]] = [dists[i], 0.0, 0.0, 0.0, v0]
    end

    params = (
    model=model,
    track=track,
    idx=idx,
    N=N,
    Q=Q,
    Qf=Qf,
    R=R,
    dt=dt,
    xi=xi,
    s_goal=s_goal,
    pathlen=pathlen,
    t_horizon=t_horizon,
    z0=z0)


    # primal bounds

    x_l = -Inf*ones(idx.nz) # update this
    x_u =  Inf*ones(idx.nz) # update this

    for i = 1:N-1
        # cross track limits
        x_l[idx.x[i+1][2]] = -5.0
        x_u[idx.x[i+1][2]] = 5.0
        
        # steering angle limits
        x_l[idx.x[i+1][4]] = -pi/4
        x_u[idx.x[i+1][4]] = pi/4
        
        # speed limits
        x_l[idx.x[i+1][5]] = 13.0
        x_u[idx.x[i+1][5]] = 13.0
        
        # steering rate limits
        x_l[idx.u[i][1]] = -2 * pi
        x_u[idx.u[i][1]] = 2 * pi

        # # accelerate/brake limits
        # x_l[idx.u[i][2]] = 0.0
        # x_u[idx.u[i][2]] = 10.0
        
    end

    # inequality constraint bounds
    c_l =  -Inf*ones(1)
    c_u =   Inf*ones(1)
    diff_type = :auto
    
    Z = fmincon(cost,equality_constraints,inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-2, c_tol = 1e-2, max_iters = 1500, verbose = false)

    # @show "dynamics constraint", dynamics_constraints(params, Z)
    
    # for i = 1:N-1
    #     print(i, "state:", Z[idx.x[i]])
    #     print("ctrl:", Z[idx.u[i]])
    #     print("\n")
    # end
    # print("final state", Z[idx.x[N]])
    # print("\n")

    return (Z, params)
end


N = 50
x = [0.0, 0, 0, 0, 13.0]
Xsim = ones(N, 5)
sim_dt = 0.2

for i = 1:N
    Z, params = optimize(X, x)
    u = Z[params.idx.u[1]]
    x = rk4(params.model, dynamics, x::Vector, u::Vector, sim_dt::Real)::Vector
    Xsim[i, :] = x
    @show x
    @show u
end



x = [2.5984542257925494, -0.04135583146744523, -0.03427559603914204, 0.02692049817878291, 13.0]
u = [0.13460249089391454, 0.0]
x = [5.191028116435372, -0.12686062839714563, -0.02482895469888914, 0.041864688871557566, 13.0]
u = [0.07472095346387328, 0.0]
x = [7.78216740861386, -0.15153413077534764, 0.008472345802550386, 0.04298749654276723, 13.0]
u = [0.005614038356048304, 0.0]
x = [10.375326988283136, -0.07685361387520175, 0.049693832549902235, 0.03776734862560212, 13.0]
u = [-0.026100739585825542, 0.0]
x = [12.968418424049156, 0.10864780745163195, 0.09340566150959873, 0.03083878589482659, 13.0]
u = [-0.03464281365387764, 1.078520768856852e-31]
x = [15.55319438078893, 0.4093098519382582, 0.1386579340805533, 0.023724936993147823, 13.0]
u = [-0.035569244508393834, -5.247404133917018e-28]
x = [18.12391777211144, 0.8240716572706631, 0.17874349116677488, 0.020186459479579005, 13.0]
u = [-0.01769238756784408, -2.916743863576693e-28]
x = [20.701304947406996, 1.31432296587167, 0.19695584042674

In [8]:
# visualization
idx = params.idx
s = 0.0:0.5:params.pathlen
idxs = get_idx_from_dist.(s)
xs = [reftraj(s[i])[1] for i = 1:length(idxs)]
ys = [reftraj(s[i])[2] for i = 1:length(idxs)]

xy_opt = [get_xy(Xsim[i, 1], Xsim[i, 2]) for i = 1:size(Xsim, 1)]
x_opt = [xy_opt[i][1] for i = 1:size(xy_opt, 1)]
y_opt = [xy_opt[i][2] for i = 1:size(xy_opt, 1)]

plot(xs, ys, ylim=(-30, 30), xlim=(0, 100),seriestype=:scatter, label="track center")
plot!(x_opt, y_opt, seriestype=:scatter, label="time-optimal trajectory")
# plot!(x_0, y_0, seriestype=:scatter, label="init traj")



LoadError: UndefVarError: `params` not defined

In [10]:
# dynamics debugging
model = (
    L = 1.2,
    lr = 0.4,
)

N = 100
x = [0.0, 0, 0, 0, 13.0]
Xsim = ones(N, 5)
sim_dt = 0.01

for i = 1:N
    u = [0.0; 0.0]
    @show "dyn", dynamics(model, x, u)
    x = rk4(model, dynamics, x::Vector, u::Vector, sim_dt::Real)::Vector
    Xsim[i, :] = x
    @show x
end

# visualization
idx = params.idx
s = 0.0:0.1:params.pathlen
idxs = get_idx_from_dist.(s)

xs = [reftraj(s[i])[1] for i = 1:length(idxs)]
ys = [reftraj(s[i])[2] for i = 1:length(idxs)]

x_opt = [Xsim[i, 1] for i = 1:size(Xsim, 1)]
y_opt = [Xsim[i, 2] for i = 1:size(Xsim, 1)]

s_0 = [params.z0[idx.x[i]][1] for i = 1:params.N]
r_0 = [params.z0[idx.x[i]][2] for i = 1:params.N]
xy_0 = get_xy.(s_0, r_0)
x_0 = [xy_0[i][1] for i = 1:params.N]
y_0 = [xy_0[i][2] for i = 1:params.N]

plot(xs, ys, ylim=(-7, 7), xlim=(0, 60),seriestype=:scatter, label="reference traj")
plot!(x_opt, y_opt, seriestype=:scatter, label="time-optimal traj")
# plot!(x_0, y_0, seriestype=:scatter, label="init traj")



("dyn", dynamics(model, x, u)) = ("dyn", [13.0, 0.0, 0.0, 0.0, 0.0])
x = [0.12999999706884263, -1.1747074318155785e-5, -0.00027930085919180143, 0.0, 13.0]
("dyn", dynamics(model, x, u)) = ("dyn", [12.99999879814944, -0.0036309111222861457, -0.059145983108166336, 0.0, 0.0])
x = [0.25999991475993856, -0.0001003835591813665, -0.0011911235451858757, 0.0, 13.0]
("dyn", dynamics(model, x, u)) = ("dyn", [12.999978408940477, -0.015484602425887204, -0.12321759924136287, 0.0, 0.0])
x = [0.3899993266832825, -0.00034920178492583607, -0.002743622489858316, 0.0, 13.0]
("dyn", dynamics(model, x, u)) = ("dyn", [12.999885673011915, -0.03566704762104714, -0.18727997168871666, 0.0, 0.0])
x = [0.519997088040717, -0.0008414808144961586, -0.004936658843332329, 0.0, 13.0]
("dyn", dynamics(model, x, u)) = ("dyn", [12.999630108024032, -0.06417630429339984, -0.2513228947744171, 0.0, 0.0])
x = [0.6499909628898877, -0.0016604713165419868, -0.007769960214080194, 0.0, 13.0]
("dyn", dynamics(model, x, u)) = ("dyn", 

LoadError: UndefVarError: `params` not defined